In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def gen_stackplot(file, output_dir='wframes', axis_step=10, colors_num=4, figsize=(14.4, 10.8)):
    assert isinstance(colors_num, int) and colors_num > 0

    testimg = mpimg.imread("frames/" + file)

    plt.figure(figsize=figsize)
    plt.subplots_adjust(0, 0, 1, 1)
    
    x = np.arange(0, testimg.shape[1], axis_step)
    if testimg.shape[1] % axis_step != 0:
        x = np.append(x, testimg.shape[1])

    color_layers = np.arange(0, 256, 255 // colors_num)
    if color_layers[-1] != 255:
        color_layers[-1] = 255

    sumlayers = np.zeros((testimg.shape[0], x.size - 1))
    for lb, rb in zip(x[:-1], x[1:]):
        for y in range(testimg.shape[0]):
            sumlayers[y, lb // axis_step] = testimg[y, lb:rb].mean()
            for lcb, rcb in zip(color_layers[:-1], color_layers[1:]):
                if rcb == 255 and sumlayers[y, lb // axis_step] >= lcb:
                    sumlayers[y, lb // axis_step] = rcb
                    break
                elif lcb <= sumlayers[y, lb // axis_step] < rcb:
                    sumlayers[y, lb // axis_step] = lcb
                    break

    layers = []
    cumlayers = [0] * sumlayers.shape[1]
    colors = []
    for i in range(sumlayers.shape[0]):
        # colors.append(sumlayers[0, 0])
        # colors_idx = len(colors)
        while min(cumlayers) <= i:
            # print(min(cumlayers))
            continue_filling = False
            colors.append(sumlayers[i, [j for j, cl in enumerate(cumlayers) if cl <= i][0]])

            # if i > 0 and not all(cl > i for cl in cumlayers):
            #     print([j for j, cl in enumerate(cumlayers) if cl <= i])
            #     print(colors[-1], sumlayers[i-1:i+2, [j for j, cl in enumerate(cumlayers) if cl <= i][0]])

            if len(colors) > 1 and colors[-1] == colors[-2]:
                continue_filling = True
                colors = colors[:-1]
            else:
                layers.append([]) 

            # if any(colors[-1] == cl for cl in colors[colors_idx: len(colors)]):
            #     colors = colors[:-1]
            #     continue

            # layers.append([])
            for j in range(sumlayers.shape[1]):
                if continue_filling:
                    # print(layers[-1][j], layers[-1])
                    if layers[-1][j] > 0 or cumlayers[j] == sumlayers.shape[0]:
                        continue
                    
                    for y in range(cumlayers[j], sumlayers.shape[0]):
                        if sumlayers[y, j] == colors[-1]:
                            layers[-1][j] += 1
                        else:
                            break
                else:
                    if sumlayers[i, j] != colors[-1] or cumlayers[j] == sumlayers.shape[0]:
                        layers[-1].append(0)
                        continue   

                    layers[-1].append(1)
                    for y in range(cumlayers[j] + 1, sumlayers.shape[0]):
                        if sumlayers[y, j] == colors[-1]:
                            layers[-1][-1] += 1
                        else:
                            break
            
            # if min(cumlayers) == 297:
                # print(*layers, sep='\n')
                # print(colors, end='\n\n')

            cumlayers = [sum(layers[i2][j2] for i2 in range(len(layers))) for j2 in range(sumlayers.shape[1])]
            # print(cumlayers)

    layers = np.array(layers)[::-1]
    # print(cumlayers)
    # print(layers[:2])
    # print(colors)

    plt.axis("off")
    plt.xlim([0, x[-2]])
    plt.ylim([0, sumlayers.shape[0]])
    plt.stackplot(x[:-1], layers, colors=np.array([[c, c, c] for c in colors])[::-1] / 255)
    
    plt.savefig(output_dir + '/' + file)
    # plt.show()
    plt.close()

# gen_stackplot("frames/output_1601.jpg", 'wframes', 5, 2)

In [ ]:
import os
from joblib import Parallel, delayed


Parallel(n_jobs=-1)(delayed(gen_stackplot)(frame, 'wframes', 5, 2) for frame in os.listdir('frames')[1:])